In [1]:
import pandas as pd
from scipy.optimize import minimize
import numpy as np
from scipy.stats import norm

In [2]:
# Define the Black-Scholes function
def black_scholes_call_price(S, K, T, r, sigma):
    """
    Compute the Black-Scholes price for a call option.
    
    Parameters:
    S - Current stock price
    K - Option strike price
    T - Time to expiration (in years)
    r - Risk-free interest rate
    sigma - Volatility
    
    Returns:
    Price of the call option under the Black-Scholes model.
    """
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    return call_price



In [3]:
def simplified_heston_price(S, K, T, r, kappa, theta, sigma, rho, v0):
    """
    Simplified Heston model pricing function.
    This is NOT the actual Heston formula but a representation for demonstration purposes.
    
    Parameters:
    S - Current stock price
    K - Option strike price
    T - Time to expiration (in years)
    r - Risk-free interest rate
    kappa, theta, sigma, rho, v0 - Heston model parameters
    
    Returns:
    Approximated price of the call option under the Heston model.
    """
    
    # Approximate the Heston volatility using the model parameters
    heston_vol = np.sqrt(v0 + kappa * (theta - v0) * T + sigma * np.sqrt(T))
    
    # Use the Black-Scholes formula with the approximated Heston volatility
    heston_price = black_scholes_call_price(S, K, T, r, heston_vol)
    
    return heston_price

In [4]:
# Load the CSV file again
csv_data = pd.read_csv('20230912_PLIQ_IP.csv')

# Extract required columns from the CSV file for Black-Scholes calculation
S = csv_data['Futuro'].values  # Current stock price (Futuro column)
K = csv_data['Serie'].values   # Option strike price (Serie column)
T = csv_data['Plazo a Vencimiento'].values  # Time to expiration (Plazo a Vencimiento column)
r = csv_data['Tasa de Interes'].values  # Risk-free interest rate (Tasa de Interes column)
sigma = csv_data['Volatilidad'].values  # Implied volatility (Volatilidad column)

# Calculate approximate call prices using the provided implied volatilities
approximated_call_prices = black_scholes_call_price(S, K, T, r, sigma)

approximated_call_prices[:10]  # Display the first 10 approximated call prices


array([18384.67725491, 17985.22266392, 17587.09672649, 17190.51453913,
       16795.28903363, 16401.45106169, 16009.01464941, 15617.97498076,
       15228.37974602, 14840.23852851])

In [5]:
# Test the simplified Heston pricing function
sample_price = simplified_heston_price(S[0], K[0], T[0], r[0], 1.5, 0.04, 0.3, -0.7, 0.04)
sample_price

17701.255821006183

In [6]:
def objective_function(params, *args):
    """
    Objective function for Heston model calibration.
    
    Parameters:
    params - Heston model parameters (kappa, theta, sigma, rho, v0)
    *args - Other arguments (S, K, T, r, market_prices)
    
    Returns:
    Sum of squared differences between market prices and Heston model prices.
    """
    
    kappa, theta, sigma, rho, v0 = params
    S, K, T, r, market_prices = args
    
    heston_prices = simplified_heston_price(S, K, T, r, kappa, theta, sigma, rho, v0)
    return np.sum((heston_prices - market_prices) ** 2)

# Initial guess for Heston parameters
initial_params = [1.5, 0.04, 0.3, -0.7, 0.04]

# Calibrate using the optimization algorithm
result = minimize(objective_function, initial_params, args=(S, K, T, r, approximated_call_prices), 
                 bounds=((0, 5), (0, 1), (0, 5), (-1, 1), (0, 1)))

# Extract calibrated parameters
calibrated_params = result.x
calibrated_params

C:\Users\Lenovo S340\AppData\Local\Temp\ipykernel_17516\1671702491.py:17: RuntimeWarning: divide by zero encountered in divide
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
C:\Users\Lenovo S340\AppData\Local\Temp\ipykernel_17516\3551741339.py:18: RuntimeWarning: invalid value encountered in sqrt
  heston_vol = np.sqrt(v0 + kappa * (theta - v0) * T + sigma * np.sqrt(T))


array([ 1.5 ,  0.04,  0.3 , -0.7 ,  0.04])

In [7]:
def implied_volatility_bs(S, K, T, r, market_price):
    """
    Calculate implied volatility using the Black-Scholes formula via a numerical method.
    
    Parameters:
    S - Current stock price
    K - Option strike price
    T - Time to expiration (in years)
    r - Risk-free interest rate
    market_price - Observed market price of the option
    
    Returns:
    Implied volatility
    """
    # Define an objective function to minimize
    def loss_function(sigma):
        return (black_scholes_call_price(S, K, T, r, sigma) - market_price) ** 2

    # Use a numerical optimizer to find the implied volatility
    result = minimize(loss_function, 0.2, bounds=[(0.001, 5)])
    return result.x[0]

# Generate smoothed option prices using the calibrated Heston parameters
smoothed_prices = simplified_heston_price(S, K, T, r, *calibrated_params)

# Convert smoothed prices to implied volatilities
smoothed_iv = np.array([implied_volatility_bs(S[i], K[i], T[i], r[i], smoothed_prices[i]) for i in range(len(S))])

smoothed_iv[:10]  # Display the first 10 smoothed implied volatilities


array([0.5050328, 0.5050328, 0.5050328, 0.5050328, 0.5050328, 0.5050328,
       0.5050328, 0.5050328, 0.5050328, 0.5050328])

In [8]:
# Prepare the data for the TXT file without the 'Tipo de Opcion' column
txt_data_smoothed = []

for idx, row in csv_data.iterrows():
    line = "{},{},{},{},{},{},{},{}".format(
        row['Fecha'], 
        row['Emisora'],
        row['Serie'],
        int(row['Pliq']),  # Convert Pliq to integer as in the provided TXT file
        round(smoothed_iv[idx], 4),  # Smoothed implied volatility
        row['Futuro'],
        row['Plazo a Vencimiento'],
        row['Tasa de Interes']
    )
    txt_data_smoothed.append(line)

# Write the data to a TXT file
output_path = r"C:\Users\Lenovo S340\Desktop\CIFI Hesston\Pruebas\20230912_PLIQ_H_IP.txt"
with open(output_path, 'w') as f:
    for line in txt_data_smoothed:
        f.write("%s\n" % line)

output_path

'C:\\Users\\Lenovo S340\\Desktop\\CIFI Hesston\\Pruebas\\20230912_PLIQ_H_IP.txt'